In [1]:
# making chatbot with bigger intents file

import nltk
import torch


nltk.download('punkt')
nltk.download('punkt_tab')


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nSetup Complete!")
print(f"PyTorch Version: {torch.__version__}")
print(f"Device Available: {device} (If 'cuda', your training will be super fast)")





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...



Setup Complete!
PyTorch Version: 2.9.0+cpu
Device Available: cpu (If 'cuda', your training will be super fast)


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
%%writefile nltk_utils.py

import nltk
import numpy as np
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()


def tokenize(sentence):
  return nltk.word_tokenize(sentence)


# a="what is your name?"
# b=tokenize(a)
# print(b)


def stem(word):
  return stemmer.stem(word.lower())


# a=['organized','organizes',"organizer"]
# b=[stem(word) for word in a]
# print(b)



def bag_of_words(tokenized_sentence,word):
  tokenized_sentence=[stem(w)for w in tokenized_sentence]
  bag=np.zeros(len(word),dtype=np.float32)


  for idx,word in enumerate(word):
    if word in tokenized_sentence:
      bag[idx]=1.0
  return bag

Overwriting nltk_utils.py


In [4]:
%%writefile model.py

import torch
import torch.nn as nn


class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(NeuralNet,self).__init__()



    self.l1=nn.Linear(input_size,hidden_size)


    self.l2=nn.Linear(hidden_size,hidden_size)


    self.l3=nn.Linear(hidden_size,num_classes)


    self.relu=nn.ReLU()


  def forward(self,x):
    out=self.l1(x)

    out=self.relu(out)


    out=self.l2(out)



    out=self.relu(out)


    out=self.l3(out)

    return out



Writing model.py


In [5]:
%%writefile train.py
import torch
import numpy as np
import torch.nn as nn
from nltk_utils import tokenize,bag_of_words,stem
from model import NeuralNet
from torch.utils.data import Dataset,DataLoader
import random
import json



with open('intents.json','r') as f:
  intents=json.load(f)


all_words=[]
tags=[]
xy=[]

for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w=tokenize(pattern)
    all_words.extend(w)
    xy.append((w,tag))



ignore_words=["?",'.',',','!']

all_words=[stem(w)for w in all_words if w not in ignore_words]
all_words=sorted(set(all_words))
tags=sorted(set(tags))




X_train=[]
y_train=[]

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


class ChatDataSet(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples


batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000


dataset = ChatDataSet()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)


        labels = labels.to(dtype=torch.long).to(device)


        outputs = model(words)


        loss = criterion(outputs, labels)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final Loss: {loss.item():.4f}')



data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)
print(f'Training complete. file saved to {FILE}')




Writing train.py


In [6]:
!python train.py

Epoch [100/1000], Loss: 1.0349
Epoch [200/1000], Loss: 0.0464
Epoch [300/1000], Loss: 0.0077
Epoch [400/1000], Loss: 0.0052
Epoch [500/1000], Loss: 0.0019
Epoch [600/1000], Loss: 0.0026
Epoch [700/1000], Loss: 0.0006
Epoch [800/1000], Loss: 0.0003
Epoch [900/1000], Loss: 0.0004
Epoch [1000/1000], Loss: 0.0002
Final Loss: 0.0002
Training complete. file saved to data.pth


In [7]:
%%writefile chat.py
import random
import json
import torch
from model import NeuralNet
from nltk_utils import bag_of_words, tokenize


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as f:
    intents = json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


bot_name = "Sam"
print("Let's chat (type 'quit' to exit)")

while True:

    sentence = input("You: ")
    if sentence == "quit":
        break




    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])


    X = torch.from_numpy(X).to(device)


    output = model(X)


    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]


    if prob.item() > 0.75:

        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Writing chat.py


In [8]:
!python chat.py

Let's chat (type 'quit' to exit)
You: hi
Sam: Hi there! Nice to see you.
You: what is your name
Sam: I am Sam, your friendly AI Assistant.
You: tell me a joke
Sam: My battery is getting low... Just kidding! 🔋
You: tell me about pakistan
Sam: It is the land of the pure, founded in 1947.
You: joke
Sam: My battery is getting low... Just kidding! 🔋
You: another joke
Sam: Why did the developer go broke? Because he used up all his cache! 
You: national animal of pakistan
Sam: The national animal of Pakistan is the Markhor (a large wild goat). 
You: national flower of pakistan
Sam: The national flower of Pakistan is Jasmine (Chambeli). 
You: who is king
Sam: He is currently one of the best batsmen in the world.
You: kaptan
Sam: He is the former PM and the 1992 World Cup winning captain.
You: imran khan
Sam: Khuda hafiz! Take care.
You: iam getting hungry
Sam: I do not understand...
You: quit
